# ETL Pipeline

The intent of this notebook is to load the 2 files which have disaster response data: messages.csv, categories.csv, clean them, combine them and load it into a SQL database.

**Sections:**
1. Load csv files 
2. Data cleaning
3. Data merge
4. Load data to SQL database

#### Import required libraries

In [56]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

### 1. Load csv files

In [57]:
def load_data(filename):
    '''
    This fucntion will load data from CSV files
    '''
    df = pd.read_csv(filename)
    return df

### 2. Data cleaning

In [58]:
def categories_cleaning(categories):
    '''
    This function will do the following:
    Split the values in the categories column on the ; character so that each value becomes a separate column. 
    Use the first row of categories dataframe to create column names for the categories data.
    Rename columns of categories with new column names.
    '''
    
    # create a dataframe of the 36 individual category columns
    categories_split = categories['categories'].str.split(';', expand=True)
    
    # select the first row of the categories dataframe
    row = categories_split.iloc[0]
    
    # use this row to extract a list of new column names for categories
    category_colnames = [each[:-2] for each in row]
    #print(category_colnames)
    
    # rename the columns
    categories_split.columns = category_colnames
          
    for column in categories_split:
        # set each value to be the last character of the string
        categories_split[column] = categories_split[column].apply(lambda x: x[-1:])
    
        # convert column from string to numeric
        categories_split[column] = categories_split[column].astype('int')
    
    categories_clean = categories.merge(categories_split, left_index=True, right_index=True, how='inner')
    
    # drop the original categories column
    categories_clean.drop(['categories'], axis=1,inplace=True)
    
    return categories_clean

### 3. Data merge

In [59]:
def data_merge(messages, categories_clean):
    '''
    This function will merge the messages and cleaned categories datasets
    '''
    df = pd.merge(messages, categories_clean,on='id')
    
    # drop duplicates
    df.drop_duplicates(inplace=True)
    
    return df

### 4. Load data to SQL database

In [60]:
def load_into_sql(df, db):
    '''
    This fucntion will load the datafame to a SQL database
    '''
    engine = create_engine(db)
    df.to_sql('DisasterMessages', engine, index=False)
    print('Data load complete!')

In [61]:
def main():
    '''
    This is the main function which calls all other functions to load data from the CSV files, clean them, merge them and
    load it back to a SQL database.
    '''
    # Load the messages dataset
    messages = load_data('messages.csv') 
    # Load the categories dataset
    categories = load_data('categories.csv') 
    
    # clean the categories dataset and split the 'categories' column into individual columns per category
    categories_clean = categories_cleaning(categories)
    
    # Merge the cleaned categories dataset and messages dataset
    df = data_merge(messages, categories_clean)
    
    # Load to a SQL database
    load_into_sql(df, 'sqlite:///DisasterMessages.db')

In [63]:
if __name__ == '__main__':
    main()

Data load complete!
